# init

> Router assembly for Phase 3 review routes

In [ ]:
#| default_exp routes.init

In [ ]:
#| export
from typing import List, Dict, Callable, Tuple

from fasthtml.common import APIRouter

from cjm_fasthtml_card_stack.core.models import CardStackUrls

from cjm_transcript_review.models import ReviewUrls
from cjm_transcript_review.routes.core import WorkflowStateStore
from cjm_transcript_review.routes.card_stack import init_card_stack_router
from cjm_transcript_review.routes.audio import init_audio_router

## Router Assembly

In [ ]:
#| export
def init_review_routers(
    state_store:WorkflowStateStore,  # The workflow state store
    workflow_id:str,  # The workflow identifier
    prefix:str,  # Base prefix for review routes (e.g., "/workflow/review")
    audio_src_url:str="",  # Audio source route (from core routes)
) -> Tuple[List[APIRouter], ReviewUrls, Dict[str, Callable]]:  # (routers, urls, routes)
    """Initialize and return all review routers with URL bundle."""
    # Create empty URL bundle (populated after routes are defined)
    urls = ReviewUrls()
    
    # Initialize card stack router
    card_stack_router, card_stack_routes = init_card_stack_router(
        state_store, workflow_id, f"{prefix}/card_stack", urls
    )
    
    # Initialize audio router
    audio_router, audio_routes = init_audio_router(
        state_store, workflow_id, f"{prefix}/audio", urls
    )
    
    # Populate the URL bundle using .to() on route functions
    urls.card_stack = CardStackUrls(
        nav_up=card_stack_routes["nav_up"].to(),
        nav_down=card_stack_routes["nav_down"].to(),
        nav_first=card_stack_routes["nav_first"].to(),
        nav_last=card_stack_routes["nav_last"].to(),
        nav_page_up=card_stack_routes["nav_page_up"].to(),
        nav_page_down=card_stack_routes["nav_page_down"].to(),
        update_viewport=card_stack_routes["update_viewport"].to(),
        save_width=card_stack_routes["save_width"].to(),
    )
    urls.audio_src = audio_src_url
    urls.speed_change = audio_routes["speed_change"].to()
    urls.toggle_auto_nav = audio_routes["toggle_auto_nav"].to()
    urls.replay_current = audio_routes["replay_current"].to()
    
    routers = [card_stack_router, audio_router]
    
    # Combine all routes for caller
    all_routes = {**card_stack_routes, **audio_routes}
    
    return routers, urls, all_routes

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()